## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
%matplotlib inline

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import gmaps.datasets
import ipywidgets as widgets
import matplotlib.pyplot as plt

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
gmaps.__version__

'0.9.0'

In [4]:
#!jupyter lab build

In [5]:
# 1. Import the WeatherPy_database.csv file.
city_data_df = pd.read_csv("/Users/jessicajohnson/Desktop/Bootcamp/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv")
city_data_df.head(10)

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lodja,CD,-3.48,23.43,70.97,88,1,1.48,clear sky
1,1,Bethel,US,41.37,-73.41,53.01,81,90,6.93,overcast clouds
2,2,Kapaa,US,22.08,-159.32,77.00,83,40,14.99,scattered clouds
3,3,Ushuaia,AR,-54.80,-68.30,57.20,33,40,19.46,shower rain
4,4,Khatanga,RU,71.97,102.50,-23.76,85,100,10.96,overcast clouds
5,5,Evensk,RU,61.95,159.23,-30.57,70,0,7.43,clear sky
6,6,Tadine,NC,-21.55,167.88,77.70,72,100,5.70,overcast clouds
7,7,Ribeira Grande,PT,38.52,-28.70,55.40,76,40,16.11,scattered clouds
8,8,Vaini,TO,-21.20,-175.20,78.80,94,20,4.70,few clouds
9,9,Cape Town,ZA,-33.93,18.42,69.01,77,0,6.93,clear sky


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 99


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"]>=min_temp)&(city_data_df["Max Temp"]<max_temp)].dropna()
preferred_cities_df.head(10)

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lodja,CD,-3.48,23.43,70.97,88,1,1.48,clear sky
2,2,Kapaa,US,22.08,-159.32,77.00,83,40,14.99,scattered clouds
6,6,Tadine,NC,-21.55,167.88,77.70,72,100,5.70,overcast clouds
8,8,Vaini,TO,-21.20,-175.20,78.80,94,20,4.70,few clouds
10,10,Rikitea,PF,-23.12,-134.97,77.83,72,30,11.30,scattered clouds
17,17,Cabo San Lucas,MX,22.89,-109.91,82.99,61,75,5.82,broken clouds
18,18,Puerto Ayora,EC,-0.74,-90.35,75.99,88,75,6.93,light intensity drizzle
20,20,Kudahuvadhoo,MV,2.67,72.89,81.00,74,57,5.32,light rain
23,23,Srivardhan,IN,18.03,73.02,73.87,57,12,4.34,few clouds
25,25,Hithadhoo,MV,-0.60,73.08,81.86,73,0,10.33,clear sky


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City ID                271
City                   271
Country                271
Lat                    271
Lng                    271
Max Temp               271
Humidity               271
Cloudiness             271
Wind Speed             271
Current Description    271
dtype: int64

In [9]:
preferred_cities_df.isnull().sum()

City ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Lodja,CD,70.97,clear sky,-3.48,23.43,
2,Kapaa,US,77.00,scattered clouds,22.08,-159.32,
6,Tadine,NC,77.70,overcast clouds,-21.55,167.88,
8,Vaini,TO,78.80,few clouds,-21.20,-175.20,
10,Rikitea,PF,77.83,scattered clouds,-23.12,-134.97,
17,Cabo San Lucas,MX,82.99,broken clouds,22.89,-109.91,
18,Puerto Ayora,EC,75.99,light intensity drizzle,-0.74,-90.35,
20,Kudahuvadhoo,MV,81.00,light rain,2.67,72.89,
23,Srivardhan,IN,73.87,few clouds,18.03,73.02,
25,Hithadhoo,MV,81.86,clear sky,-0.60,73.08,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        pass
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Lodja,CD,70.97,clear sky,-3.48,23.43,
2,Kapaa,US,77.00,scattered clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
6,Tadine,NC,77.70,overcast clouds,-21.55,167.88,
8,Vaini,TO,78.80,few clouds,-21.20,-175.20,Keleti Beach Resort
10,Rikitea,PF,77.83,scattered clouds,-23.12,-134.97,Pension Maro'i
17,Cabo San Lucas,MX,82.99,broken clouds,22.89,-109.91,Hotel Tesoro Los Cabos
18,Puerto Ayora,EC,75.99,light intensity drizzle,-0.74,-90.35,Finch Bay Galapagos Hotel
20,Kudahuvadhoo,MV,81.00,light rain,2.67,72.89,Kudahuvadhoo Retreat
23,Srivardhan,IN,73.87,few clouds,18.03,73.02,Savli Resort/Hotel - Shrivardhan
25,Hithadhoo,MV,81.86,clear sky,-0.60,73.08,Scoop Guest House


In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= ""].dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Kapaa,US,77.00,scattered clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
8,Vaini,TO,78.80,few clouds,-21.20,-175.20,Keleti Beach Resort
10,Rikitea,PF,77.83,scattered clouds,-23.12,-134.97,Pension Maro'i
17,Cabo San Lucas,MX,82.99,broken clouds,22.89,-109.91,Hotel Tesoro Los Cabos
18,Puerto Ayora,EC,75.99,light intensity drizzle,-0.74,-90.35,Finch Bay Galapagos Hotel
20,Kudahuvadhoo,MV,81.00,light rain,2.67,72.89,Kudahuvadhoo Retreat
23,Srivardhan,IN,73.87,few clouds,18.03,73.02,Savli Resort/Hotel - Shrivardhan
25,Hithadhoo,MV,81.86,clear sky,-0.60,73.08,Scoop Guest House
26,Tahoua,NE,72.81,clear sky,14.89,5.27,Bungalows Guesthouse
30,Betare Oya,CM,76.93,broken clouds,5.60,14.08,Auberge Tembina Betaré Oya


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


In [18]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Save the figure
#plt.savefig("WeatherPy_vacation_map.png")

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))